In [6]:
import plotly.express as px
import pandas as pd
import numpy as np

from scipy import optimize
from functools import partial

## Virtual Population Analysis

The basic idea here is pretty simple. Given some age group $N_{t}$, that age group's survival will entirely determine the following year's age group $N_{t-1}$. In other words by modeling a specific age group $t>1$ we don't have to worry about reproduction or recruitment at all. 

First we start with an equation from our Growth and Mortality notebook:

$$N_{t+1} = N_{t} e^{-(M + F_t)}$$

From this we know that:

$$C_t = \frac{F_t}{F_t + M}N_t (1-e^{-(M + F_t)})$$

Which we can rearrange as:

$$N_t=\frac{C_t(F_t + M)}{F_t(1-e^{-(M + F_t)})}$$

This means that given an estimate of $M$, $F_t$, and $C_t$ we can compute an estimate of $N_t$.

Then we can note that:

$$\frac{N_{t+1}}{C_t}=\frac{(F_t + M)e^{-(M + F_t)}}{F_t(1-e^{-(M + F_t)})}$$

Which means we can solve for $F_{t-1}$ using $M$, $C_{t-1}$, and $N_{t}$. Then once $F_{t-1}$ is known we can use the following to solve for $N_{t-1}$:

$$N_{t-1} = N_te^{M + F_{t-1}}$$

And all in all this gives us a way for recursively solving for the survival of the population over time given its present mortality and catch and historic catch amounts.

In [39]:
def evaluate_F(M, last_N_t, C_t, F_t):
    return abs(
        (last_N_t/C_t)
        - (
            (
                (F_t + M) * np.exp(-(F_t + M))
            ) / (
                F_t * (1 - np.exp(-(M + F_t)))
            )
        )
    )

def build_history(M, F_t, C, A):
    # first we estimate the biomass at the 
    # begining of the last year
    C_t = C[0]
    N_t = (
        C_t * (F_t + M)
    ) / (
        F_t * (1 - np.exp(-(M + F_t)))
    )
    N = [N_t]
    F = [F_t]
    for C_t in C[1:]:
        last_N_t = N_t
        last_F_t = F_t
        F_t = optimize.minimize(
            partial(evaluate_F, M, last_N_t, C_t), [last_F_t], method='Nelder-Mead'
        ).x[0]
        N_t = last_N_t * np.exp(M + F_t)
        N.append(N_t)
        F.append(F_t)
    return pd.DataFrame(
        np.array([A, C, N, F]).T,
        columns=['age', 'catch', 'number', 'F']
    ).sort_values('age', ascending=True)

build_history(M=0.2, F_t=0.4, C=np.array([250, 300, 200, 50]), A=np.array([4, 3, 2, 1]))
    

,age,catch,number,F
3,1.0,50.0,2330.046392,0.023952
2,2.0,200.0,1862.531407,0.125821
1,3.0,300.0,1344.625229,0.281074
0,4.0,250.0,831.138456,0.400000


## Yield Per Recruit Models

Yield per recruit models attempt to understand what would happen if fishing patterns changed to be more selective about which age groups to capture. As we've seen before allowing the fish to grow up a bit can be really helpful in maximizing yields, especially when those fish don't have high mortality. However these yield per recruit models have several assumptions built into them:

### Stable Demographics

The first assumption is that the population stays the same size across all age groups as the years go by. This allows us to simplify our lives by recognizing that the catch from a single year is equal to the catch of a single cohort over its full "lifespan". So in order to evaluate the catch per year, we look at the catch per cohort instead (i.e. the yield per recruit idea). 

### Perfectly Selective Gear

The next thing these models assume is that the gear *never* catches anything below a specific age. This is a sort of "knife's edge" assumption (which is quite unrealistic). 

### von Bertalanffy growth curves

This one is pretty self explanatory. 

### Constant Mortality

This assumes a constant $M$ and $F$ (where applicable)

Alright so how does all of this come together? Well first of all we deem the number at recruitment as $N_{t_r}$ ($t_r$ is the age at recruitment). Then the number at $t_c$ (age when the gear starts catching these fish) as:

$$N_{t_c}=N_{t_r}e^{-M(t_c - t_r)}$$

and then at subsequent ages:

$$N_t = N_{t_c}e^{-(M + F)(t - t_c)}=N_{t_r}e^{-M(t_c - t_r)}e^{-(M + F)(t - t_c)}$$

Then the yield per recruit is just gained by integrating this (converted into weight) over time with a $N_{t_r}=1$

Technically in the book yield per recruit is given for the first year of capture only. 